In [1]:
#Compressing file using Huffman Code

import heapq
import os

class BinaryTreeNode:
    
    def __init__(self, key, frequency):
        self.key=key
        self.freq=frequency
        self.left=None
        self.right=None
        
    def __lt__(self, other):
        
        return self.freq < other.freq
    
    def __eq__(self, other):
        
        return self.frequency==other.frequency
    

class HuffmanCode:
    
    def __init__(self, path):
        self.path=path
        self.__heap=[]
        self.__codes={}
        self.__reverse_codes={}
        
    def __getBytesArray(self, padded_encoded_text):
        
        array=[]
        for i in range(0,len(padded_encoded_text),8):
            byte=padded_encoded_text[i:i+8]
            array.append(int(byte, 2))
        return array
        
    def __paddedEncodedText(self, encoded_text):
        
        paddingAmount=8-(len(encoded_text)%8)
        for i in range(paddingAmount):
            encoded_text+="0"
        padded_info="{0:08b}".format(paddingAmount)
        padded_encoded_text=padded_info+encoded_text
        return padded_encoded_text
        
    def __getEncodedText(self, text):
        
        encoded_text=""
        for char in text:
            encoded_text+=self.__codes[char]
        return encoded_text
        
    def create_frequency_dict(self, text):
        
        freq_dict={}
        for char in text:
            if char not in freq_dict:
                freq_dict[char]=0
            freq_dict[char]+=1
        return freq_dict
    
    def __buildHeap(self, freq_dict):
        
        for key in freq_dict:
            frequency=freq_dict[key]
            binary_tree_node=BinaryTreeNode(key, frequency)
            heapq.heappush(self.__heap, binary_tree_node)
            
    def __buildTree(self):
        
        while (len(self.__heap)>1):
            binary_tree_node_1=heapq.heappop(self.__heap)
            binary_tree_node_2=heapq.heappop(self.__heap)
            freq_sum=binary_tree_node_1.freq+binary_tree_node_2.freq
            newNode=BinaryTreeNode(None, freq_sum)
            newNode.left=binary_tree_node_1
            newNode.right=binary_tree_node_2
            heapq.heappush(self.__heap, newNode)
        return
    
    def __buildCodesHelper(self, root, curr_bits):
        
        if root is None:
            return
        if root.key is not None:
            self.__codes[root.key]=curr_bits
            self.__reverse_codes[curr_bits]=root.key
            return
        
        self.__buildCodesHelper(root.left, curr_bits+'0')
        self.__buildCodesHelper(root.right, curr_bits+'1')
    
    def __buildCodes(self):
        
        root=heapq.heappop(self.__heap)
        self.__buildCodesHelper(root, "")
        return
    
    def compress(self):
        
        #get file from the path
        file_name, file_extension=os.path.splitext(self.path)
        output_path=file_name+'.bin'
        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:
            
            #read the file    
            text=file.read()
            #text=text.rstrip()
            
            #create frequency dictionary
            freq_dict=self.create_frequency_dict(text)
            
            #create min priority heap from freq_dict
            self.__buildHeap(freq_dict)
            
            #construct binary tree from heap
            self.__buildTree()
            
            #construct the codes from binary tree
            self.__buildCodes()
            
            #encode the text usinf huffman codes we obtained from the binary tree
            encoded_text=self.__getEncodedText(text)
            
            #pad the encoded text
            padded_encoded_text=self.__paddedEncodedText(encoded_text)
            
            #creating byte array
            byte_array=self.__getBytesArray(padded_encoded_text)
            
            #text is coverted into bytes
            final_bytes=bytes(byte_array)
            
            #writing in the output file
            output.write(final_bytes)
            
        print('!!Compression Successfull!!')
        #return binary file as output
        return output_path
    
    def __removePadding(self, text):
        padded_info=text[:8]
        padded_size=int(padded_info, 2)
        actual_text=text[8:-1*padded_size]
        return actual_text
    
    def __decodeText(self, text):
        current_bits=''
        decoded_text=''
        for bit in text:
            current_bits+=bit
            if current_bits in self.__reverse_codes:
                decoded_text+=self.__reverse_codes[current_bits]
                current_bits=''
        return decoded_text
        
    def decompress(self, input_path):
        file_name,file_extension=os.path.splitext(self.path)
        output_path=file_name+'_compressed'+'.txt'
        with open(input_path, 'rb') as file, open (output_path, 'w') as output:
            bit_string=""
            byte=file.read(1)
            while byte:
                byte=ord(byte)
                bits=bin(byte)[2:].rjust(8,'0')
                bit_string+=bits
                byte=file.read(1)
                
            #removing padded text from the actual text
            actual_text=self.__removePadding(bit_string)
            
            # decoding the actual text
            decompressed_text=self.__decodeText(actual_text)
            
            #writing in the output file
            output.write(decompressed_text)
            print('!! Decompression Successful !!')
            
        return
#main
print('Enter path of the file to compress: ')
path=input()
h=HuffmanCode(path)
output_path=h.compress()
print("Do you want to decompress the file (press Y) ?")
response=input()
if response.upper()=='Y':
    h.decompress(output_path)
else:
    print('Exiting Program')
        

Enter path of the file to compress: 
/home/dell/Desktop/sample
!!Compression Successfull!!
Do you want to decompress the file (press Y) ?
y
!! Decompression Successful !!
